In [1]:
import pandas as pd

In [2]:
def prepravi_dp(data_point:dict) -> dict:
    resenje = {}
    resenje['tokens'] = [x[0] for x in data_point['tokens']]

    broj_tokena = len(resenje['tokens'])
    tags = ['O' for _ in range(broj_tokena)]

    for entity in data_point.get('entities',[]):
        start = entity.get('token_start', -1)
        end = entity.get('token_end', -1)
        tip = entity.get('type','O')
        # Provera granica
        if start < 0 or end < 0 or end >= broj_tokena:
            continue

        tags[start] = f'B-{tip}'
        tags[start + 1:end + 1] = [f'I-{tip}'] * (end - start)

    resenje['tags'] = tags
    return resenje

In [3]:
def ucitaj_u_nerFormatu(folder_path, lista = False)->list:
    import os
    import json
    from glob import glob

    all_data = []
    if not lista:
        json_files = glob(os.path.join(folder_path, '*.json'))
    else:
        json_files = folder_path
        
    for file_path in json_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                json_data = json.load(file)
            
            for data_point in json_data:
                entities = data_point.get('entities', [])
                
                row = {
                    'id': data_point.get('id'),
                    'text': data_point.get('text'),
                    'entities': entities, 
                    'tokens': data_point.get('tokens'),
                    'token_validity': data_point.get('token_validity')
                }
                
                # Dodajemo red u listu svih podataka
                all_data.append(prepravi_dp(row))
        except Exception as e:
            print(f"Greška prilikom obrade fajla {file_path}: {e}")
    return all_data
# Kreiranje DataFrame-a od svih podataka


In [4]:
def sacuvaj_nerFormat(data, putanja):
    import json

    with open(putanja, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=1, ensure_ascii=False)

In [5]:
folder_path = '../anotations/remaining_set'
data = ucitaj_u_nerFormatu(folder_path)
sacuvaj_nerFormat(data, '../anotations/ner_data/train.json')
df = pd.DataFrame(data)
df.iloc[0]

tokens    [“, You, can, go, first, ,, ”, Ron, snarled, ....
tags      [O, O, O, O, O, O, O, B-CHARACTER, O, O, O, O,...
Name: 0, dtype: object

In [6]:
df.head()

,tokens,tags
0,"[“, You, can, go, first, ,, ”, Ron, snarled, ....","[O, O, O, O, O, O, O, B-CHARACTER, O, O, O, O,..."
1,"[Harry, took, the, wa, ##nd, and, (, feeling, ...","[B-CHARACTER, O, O, B-MAGIC_ITEM, I-MAGIC_ITEM..."
2,"[', Oh, ,, they, only, know, flash, ##y, stuff...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[', You, know, that, was, a, flu, ##ke, ,, if,...","[O, O, O, O, O, O, O, O, O, O, O, B-MAGIC_ITEM..."
4,"[“, Yeah, ,, and, that, ', s, not, a, stupid, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [19]:
# sada priprema val i test!
folder_path = '../anotations/golden_set_checked'
data = ucitaj_u_nerFormatu(folder_path)

import random 
random.shuffle(data)

test = data[:100]
val = data[100:]


In [ ]:
folderi = ['../anotations/golden_set/golden_set_part_006.json', '../anotations/golden_set/golden_set_part_010.json', '../anotations/golden_set/golden_set_part_011.json']

ner_data = ucitaj_u_nerFormatu(folderi,True)
test+=ner_data 

175


In [21]:
sacuvaj_nerFormat(test, '../anotations/ner_data/test.json')
sacuvaj_nerFormat(val, '../anotations/ner_data/val.json')